### imports and functions

In [ ]:

import pandas as pd
import numpy as np
import os


# Get Ordered transaction data for top 150 consumers

In [2]:
transactions_data_path="kaggle_data/enhanced_transactions.csv"

In [3]:
trans_columns=['customer_id','t_dat','article_id','product_type_name']

In [4]:
trans=pd.read_csv(transactions_data_path,usecols = trans_columns)

In [5]:
trans=trans.reset_index()

In [7]:
trans['customer_order_number'] = (trans[['customer_id','t_dat','article_id','index']].sort_values(['t_dat','index'])
                      .groupby(['customer_id']).cumcount(ascending=False)+1
                    )

In [8]:
trans=trans.drop(['index'], axis=1)

In [9]:
trans

,t_dat,customer_id,article_id,product_type_name,customer_order_number
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,Underwear body,18
1,2018-09-20,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,663713001,Underwear body,126
2,2018-09-20,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,663713001,Underwear body,340
3,2018-09-20,6b7b10d2d47516c82a6f97332478dab748070f09693f09...,663713001,Underwear body,109
4,2018-09-20,8ac137752bbe914aa4ae6ad007a9a0c5b67a1ab2b2d474...,663713001,Underwear body,2
...,...,...,...,...,...
31788319,2020-09-22,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,860135008,Shirt,3
31788320,2020-09-22,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,539060028,Hoodie,2
31788321,2020-09-22,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,539060028,Hoodie,1
31788322,2020-09-22,ee9c048de16034be4aa95c91adce7f7bda442e6528406e...,810737015,Sleeping sack,1


In [10]:
first_data_customers=trans.groupby(['customer_id']).agg({'customer_order_number':'count'}).sort_values(by=['customer_order_number'],ascending=False).head(100).reset_index()



In [11]:
first_data_customers.head()

,customer_id,customer_order_number
0,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,1895
1,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,1441
2,49beaacac0c7801c2ce2d189efe525fe80b5d37e46ed05...,1364
3,a65f77281a528bf5c1e9f270141d601d116e1df33bf9df...,1361
4,cd04ec2726dd58a8c753e0d6423e57716fd9ebcf2f14ed...,1237


In [12]:
merged_df = pd.merge(trans, first_data_customers['customer_id'], on='customer_id', how='inner')
selected_rows = merged_df[trans.columns]

In [13]:
first_one_fifty_products=selected_rows[selected_rows['customer_order_number']<=150].sort_values(by=['customer_id','customer_order_number'])

In [14]:
initial_data_set=first_one_fifty_products.reset_index(drop=True).pivot(index='customer_id', columns='customer_order_number', values='product_type_name')

In [15]:
initial_data_set

customer_order_number,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
customer_id,,,,,,,,,,,,,,,,,,,,,
03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296,Cardigan,Cardigan,Cardigan,Blouse,Blouse,Trousers,Trousers,Blouse,Shirt,Trousers,...,Skirt,Trousers,Trousers,Trousers,Shorts,Leggings/Tights,Skirt,Skirt,Blouse,Belt
03fdb0bf2d9ff8ba23e1b4aef53709119aad5bc83691d89293a01a52b93d7370,Trousers,Trousers,Trousers,Trousers,Trousers,Trousers,Sweater,T-shirt,Sweater,Sweater,...,Dress,Skirt,Skirt,Sweater,Skirt,Skirt,Skirt,Dress,Swimwear bottom,Swimwear bottom
06d23b72cac134851a761473b40c881c300814981b6b0a3c4692ddf5a93f2a48,Trousers,Bag,Dress,Bra,Top,Bra,Earring,Trousers,Top,Trousers,...,Jacket,Jacket,Dress,Dress,Top,Skirt,Top,Bra,Top,Top
0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54,Blouse,Blouse,Blouse,Blouse,Blouse,Blouse,Dress,Top,Blouse,Dress,...,Bag,Bra,Swimwear bottom,Bikini top,Bra,Skirt,Bra,Bikini top,Trousers,Swimwear bottom
0d4fb6fb46dfe2759bcf7bc80340e8915b207aa2f74b5b3c76f74d3ce28359e8,Trousers,Bag,Bag,Jacket,Dress,Leggings/Tights,Underwear body,Trousers,Shorts,Jumpsuit/Playsuit,...,Top,Skirt,Top,Top,Underwear body,Bra,Bikini top,Dress,Dress,Swimwear bottom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
efaafb08a00e63ce561a67c31c1ab7e720d4f394a78c4705518dc02084a29172,Jacket,Jacket,Top,Top,Top,Trousers,Trousers,Trousers,Sweater,Top,...,Wedge,Sweater,Leggings/Tights,Sneakers,Trousers,Trousers,Trousers,Dress,Sweater,Sweater
f0fb2d661e4b96343c84b350878cdd983be790e946c16d44315ac0d548573046,Shorts,Shorts,Shorts,Shorts,Shorts,Shorts,Dress,Dress,Blouse,Blouse,...,Jumpsuit/Playsuit,Jumpsuit/Playsuit,Dress,Dress,Bra,Hoodie,Dress,Dress,Night gown,Night gown
f137c16fd175271922dad4006565503952f24750a57388fe24970a218c62de6a,Ballerinas,Trousers,Blouse,Top,Cardigan,Cardigan,Cardigan,Cardigan,Shirt,Shirt,...,Vest top,Vest top,Blouse,Trousers,Vest top,Trousers,Trousers,Vest top,Shirt,Blouse


In [16]:
path='kaggle_data/'
initial_data_set.to_csv(os.path.join(path,r'initial_data_set.csv'),index=False)